<a href="https://colab.research.google.com/github/hanbiphyun/ESSA_YB/blob/main/ESAA_YB_week12_%EC%88%98%EC%83%81%EC%9E%91%EB%A6%AC%EB%B7%B03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

링크 : https://dacon.io/competitions/official/236439/codeshare/12230

##부동산 허위매물 분류 프로젝트

###1. 프로젝트 개요

본 작업은 부동산 플랫폼에 등록된 매물 정보를 기반으로 **해당 매물이 허위매물인지 여부(0: 실매물, 1: 허위매물)** 를 예측하는 이진 분류 문제를 해결하는 과정 전반을 분석한 내용이다. 사용된 데이터는 총 2452개 행, 17개 컬럼으로 구성되어 있으며, 매물의 보증금·월세·전용면적·층수·방향 등 다양한 정형 데이터로 이루어져 있다.

###2. 데이터 구성

데이터는 아래와 같은 주요 변수로 구성되어 있다.

ID

매물확인방식

보증금

월세

전용면적

해당층

총층

방향

방수

욕실수

주차가능여부

총주차대수

관리비

중개사무소

제공플랫폼

게재일

허위매물여부 (타겟)


###**3. 데이터 전처리**

3.1 결측치 처리

- 수치형 변수는 평균값으로 대체하였다.

- 범주형 변수는 추후 레이블 인코딩을 위해 원본 그대로 유지하였다.

3.2 이상치 제거 (DBSCAN)

- DBSCAN은 밀도 기반 클러스터링 알고리즘으로, 주변에 충분한 데이터가 존재하지 않는 점들을 노이즈(-1)로 분류한다.

- 사용한 주요 설정:

표준화(StandardScaler) 적용

eps = 2.5

min_samples = 5

- DBSCAN을 적용한 결과, 노이즈로 판단된 7개 행을 제거하여 2452 → 2445개 데이터로 정제하였다.

이유:

부동산 데이터는 보증금, 월세 등 값의 범위가 매우 넓어 극단치(outlier)가 많다.

거리 기반 이상치 탐지는 스케일링을 통해 단위를 맞춰야 효과적으로 작동한다.

DBSCAN은 비선형·비정규 데이터에 강하며 복잡한 이상치 패턴을 탐지하는 데 적합하다.

###**4. 범주형 변수 처리 (Label Encoding)**
4.1 훈련 데이터 인코딩

- 범주형 변수: 중개사무소, 제공플랫폼, 방향, 매물확인방식, 주차가능여부

- 컬럼별로 LabelEncoder 객체를 따로 생성하여 각 범주의 정수 매핑을 학습한다.

- 모든 인코더를 딕셔너리에 저장한다.

4.2 테스트 데이터 인코딩

- train에서 학습된 인코더를 그대로 사용해야 데이터 누수가 발생하지 않는다.

- test 데이터에 훈련 데이터에 없던 새로운 범주가 존재할 경우,
classes_ 에 해당 범주를 추가한 후 transform을 진행한다.

- 이는 실제 서비스 환경에서 매우 중요한 처리 방식이다.

###**5. 다중공선성 확인**

- Heatmap과 Variance Inflation Factor(VIF)를 활용해 변수 간 상관성을 검사하였다.

- 상관계수는 최대 약 0.3 수준으로 낮았으며,
VIF 역시 10을 넘는 변수는 없었으므로 변수를 제거하지 않았다.

###**6. 모델 학습**
6.1 XGBoost Classifier 사용

- 기본 모델:
XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

6.2 RandomizedSearchCV를 통한 하이퍼파라미터 탐색
- 탐색한 주요 파라미터 범위:

n_estimators

learning_rate

max_depth

min_child_weight

subsample

colsample_bytree

gamma

reg_alpha

reg_lambda

scale_pos_weight

max_delta_step

설정:

n_iter = 50

cv = 5

평가 지표: f1_macro

n_jobs = -1

- RandomizedSearchCV는 GridSearch보다 훨씬 적은 연산으로 넓은 탐색 공간을 빠르게 점검할 수 있다는 장점이 있다.

###**7. 인사이트**
7.1 얻은 인사이트

1. DBSCAN 기반 이상치 제거 전략
부동산 데이터는 연속형 변수의 범위가 넓고 극단치가 많아 단순 IQR 방식보다 DBSCAN이 효과적이었다.

2. 범주형 인코딩에서의 데이터 누수 방지 처리
LabelEncoder 사용 시 train과 test 간 매핑 불일치 문제를 적절히 해결했다.

3. RandomizedSearchCV의 효율성
모든 조합을 탐색하는 GridSearch보다 훨씬 빠르고 실용적이다.

7.2 수상작의 보완점

1. Feature Engineering이 거의 수행되지 않았으며, 도메인 기반 피처 추가 여지가 많다.

2. XGBoost 단일 모델만 사용했는데,
LightGBM, CatBoost, Stacking 등을 사용하면 성능 향상을 기대할 수 있다.